In [1]:
# https://github.com/gcdart/MulticlassClassifier/blob/master/src/ml/LogisticRegression.java
# https://www.kaggle.com/c/lshtc/discussion/6911#38233 - preprocessing: multilabels comma should not have spaces
# https://www.kaggle.com/c/lshtc/discussion/14048 - dataset statistics
## reading the LWIKI, SWIKI dataset

In [2]:
import os
import numpy as np
import pandas as pd
import igraph as ig

from pathlib import Path
from tqdm import tqdm #always use this instead of `import tqdm`
# from sklearn.datasets import fetch_rcv1

# np.random.seed(123)

In [3]:
import logging
from collections import OrderedDict

logging.basicConfig(level=logging.INFO)

In [4]:
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing, metrics

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from joblib import Memory
from sklearn.datasets import load_svmlight_file

In [7]:
mem = Memory("./mycache")
@mem.cache
def get_data(filename):
    
    fname = str(Path(filename))
    fe, ex = os.path.splitext(fname) 

    try:
        data = load_svmlight_file(fname, multilabel=True)
    except:
        # Required: if the input data isn't in the correct libsvm format
        outfile = str(Path("{}_small{}".format(fe, ex)))
#         outfile = str(Path("{}_remapped{}".format(fe, ex)))
        if not os.path.isfile(outfile):
            logging.info("Remapping data to LibSVM format...")
            f = preprocess_libsvm(fname, outfile)
        else:
            logging.info("Using already remapped data...")
            f = outfile
        data = load_svmlight_file(f, multilabel=True)
        
    return data[0], data[1]

In [8]:
def preprocess_libsvm(input_file, output_file):
    # converts file to the required libsvm format.
    # this is very brute force but can be made faster [IMPROVE]

    file = open(output_file, "w+")
    with open(input_file, "r") as f:
        head = [next(f) for x in range(500)] # retrieve only `n` docs
        for i, line in enumerate(tqdm(head)): # change to f/head depending on your needs
            instance = line.strip().split()
            labels = instance[0]
            doc_dict = OrderedDict()
            temp_dict = {}
            temp_string = ''

            for pair in instance[1:]:
                feat = pair.split(":")
                if int(feat[0]) not in temp_dict:
                    temp_dict[int(feat[0])] = int(feat[1])

            for key in sorted(temp_dict.keys()):
                doc_dict[key] = temp_dict[key]

            for feat, tf in doc_dict.items():
                temp_string = temp_string + "{}:{} ".format(feat, tf)        
            file.write("{} {}\n".format(labels, temp_string))
        file.close()

    return output_file

In [9]:
def label_extractor(labels):

    leaf_labels = set()
    labels_per_doc = []

    for i in labels:
        labels_per_doc.append(len(i))
        for j in i:
            leaf_labels.add(int(j))
    
    return leaf_labels, labels_per_doc

In [10]:
def read_hier(filename):
    
    N = set()
    pi = set()
    T = set()
    
    with open(filename, "r") as f:
        for i, line in enumerate(f):
            words = line.strip().split()
            pi.add(int(words[0])) #adding parent node
            T.add(int(words[-1])) #adding ALL leaf nodes in the hierarchy
            for w in words:
                N.add(int(w))

    return N, pi, T

In [11]:
mem = Memory("./mycache")
@mem.cache
def rr_reader(filename):
    '''
    create a dataframe from the data-label pair
    '''

    num_entries = 200000
    df = pd.DataFrame()
    
    with open(filename, "r") as f:
        head = [next(f) for x in range(num_entries)] # retrieve only `n` docs
        for i, line in enumerate(tqdm(head)): # change to f/head depending on your needs
            instance = line.strip().split()
            labels = instance[0]
            doc_dict = OrderedDict()
            temp_dict = {}

            for pair in instance[1:]:
                feat = pair.split(":")
                if int(feat[0]) not in temp_dict:
                    temp_dict[int(feat[0])] = int(feat[1])

            for key in sorted(temp_dict.keys()):
                doc_dict[key] = temp_dict[key]
                
            temp_df = pd.DataFrame(data = [ labels, doc_dict ]).T
            df = df.append(temp_df, ignore_index=True)
    
    df.columns = ["labels", "feat_tf"]
    df["labels"] = df["labels"].apply( lambda x: list(map(int, x.split(",")))  )
    return df

In [12]:
df = rr_reader("swiki/data/train.txt")

In [13]:
Nn, pii, Ti = read_hier("swiki/data/cat_hier.txt")

In [14]:
d = df

In [15]:
# whatiwant = [14661, 71999, 292915, 188756, 131368, 130762, 352578, 395447, 27512, 157031, 33692, 13402, 393382, 390846, 395447, 276114]
whatiwant = [14661, 71999, 292915, 188756, 131368]

In [16]:
for j in whatiwant:
    d[str(j)] = 0

In [17]:
d.head()

,labels,feat_tf,14661,71999,292915,188756,131368
0,"[33692, 13402, 393382]","{624: 3, 4288: 1, 14403: 1, 54278: 1, 62619: 1...",0,0,0,0,0
1,[130762],"{120505: 1, 173442: 1, 554009: 1, 634374: 1, 6...",0,0,0,0,0
2,"[352578, 395447, 27512, 157031]","{62483: 1, 73429: 1, 138155: 1, 160218: 1, 165...",0,0,0,0,0
3,"[390846, 395447, 276114]","{200187: 1, 207596: 1, 343448: 1, 359544: 1, 4...",0,0,0,0,0
4,"[14661, 71999, 292915, 188756, 131368]","{2906: 1, 4288: 1, 17471: 3, 56146: 1, 94588: ...",0,0,0,0,0


In [18]:
for i in range(len(d)):
    for j in whatiwant:
        if j in d.loc[i, "labels"]:
            d.loc[i, str(j)] = 1

In [19]:
# num of docs per label (id)
for j in whatiwant:
    print(j, sum(d[str(j)]))

14661 482
71999 99
292915 39
188756 43
131368 41


In [20]:
subset = pd.DataFrame()

In [21]:
subset = d[d["14661"]==1]

In [22]:
subset.sample(n=5)

,labels,feat_tf,14661,71999,292915,188756,131368
148742,"[27265, 14661, 415589, 92007]","{16441: 4, 26109: 1, 92422: 1, 94588: 5, 11183...",1,0,0,0,0
167059,"[216803, 14661, 164163]","{8032: 1, 78034: 1, 92366: 1, 110218: 7, 14874...",1,0,0,0,0
84060,"[347803, 14661]","{183769: 1, 236566: 1, 435388: 1, 474135: 1, 4...",1,0,0,0,0
192027,"[216803, 14661, 127318]","{624: 1, 4023: 1, 8032: 1, 11228: 2, 19335: 1,...",1,0,0,0,0
102733,"[14661, 347803, 10590]","{24239: 1, 25642: 1, 32119: 1, 33751: 1, 35704...",1,0,0,0,0


In [23]:
train_data, _ = get_data("swiki/data/train_remapped.txt")

In [24]:
max(list(subset.index))

199594

In [25]:
ll = []
for i in list(subset.index):
    try:
        ll.append(train_data[int(i)].toarray())
    except:
        print(i)

In [26]:
niu = np.concatenate( ll, axis=0 )

In [27]:
niu.shape

(482, 2085164)

In [28]:
train_x, train_y, test_x, test_y = train_test_split(niu, subset, random_state=42, test_size=0.30, shuffle=False)

In [29]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((337, 2085164), (145, 2085164), (337, 7), (145, 7))

In [30]:
x_train = train_x
y_train = test_x.drop(labels = ["labels", "feat_tf"], axis = 1)

x_test = train_y
y_test = test_y.drop(labels = ["labels", "feat_tf"], axis = 1)

In [31]:
y_train.sample(n=5)

,14661,71999,292915,188756,131368
98512,1,0,0,0,0
79156,1,0,0,0,0
129243,1,0,0,0,0
42555,1,0,0,0,0
91192,1,0,0,0,0


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

In [33]:
clf = SGDClassifier()
# lb = preprocessing.LabelBinarizer(sparse_output=True)
# mb = preprocessing.MultiLabelBinarizer(sparse_output=True)
# le = preprocessing.LabelEncoder()

In [34]:
# SGD Classifier
for category in whatiwant[1:]:
    print('**Processing {} tag...**'.format(category))
    
    # Training sgd model on train data
    clf.fit(x_train, y_train[str(category)])
    
    # calculating test accuracy
    prediction = clf.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test[str(category)], prediction)))
    print("\n")

**Processing 71999 tag...**


c:\users\harshasivajit\documents\master-ai\venv\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Test accuracy is 0.993103448275862


**Processing 292915 tag...**


c:\users\harshasivajit\documents\master-ai\venv\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Test accuracy is 1.0


**Processing 188756 tag...**


c:\users\harshasivajit\documents\master-ai\venv\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Test accuracy is 1.0


**Processing 131368 tag...**


c:\users\harshasivajit\documents\master-ai\venv\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Test accuracy is 1.0




In [35]:
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
for category in whatiwant[1:]:
    print('**Processing {} tag...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, y_train[str(category)])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test[str(category)], prediction)))
    print("\n")

**Processing 71999 tag...**


c:\users\harshasivajit\documents\master-ai\venv\lib\site-packages\sklearn\externals\joblib\disk.py:122: UserWarning: Unable to delete folder C:\Users\HARSHA~1\AppData\Local\Temp\joblib_memmapping_folder_9996_6487582555 after 5 tentatives.
  .format(folder_path, RM_SUBDIRS_N_RETRY))


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\HARSHA~1\\AppData\\Local\\Temp\\joblib_memmapping_folder_9996_6487582555\\9996-2618438635304-5c82b6b12b82470f8bc2a021c02a4da5.pkl'

In [36]:
print(len(Nn), len(Ti)) # there is one node without any parent
print(Nn.difference(Ti)) # this is the node: this is probably the root node i guess

50312 50311
{2143406}


In [37]:
len(pii)

13808

In [38]:
 def lookup_table(filename, subset):
        
    p2c_table = {}
    c2p_table = {}
    node2id = OrderedDict()
    id2node = OrderedDict()
    i = 0
    
    with open(filename, "r") as f:
        if not subset:
            head = f
        elif isinstance(subset, int):
            head = [next(f) for x in range(subset)] # retrieve only `n` docs
        else:
            raise ValueError("Incorrect subset type. Enter only False (boolean) or int. Encountered {} type.".format(type(subset)))
        for _, line in enumerate(tqdm(head)):
            split_line = line.strip().split()
            parent_node = int(split_line[0])
            child_node = list(map(int, split_line[1:]))
            
            # map to the respective dicts -> parent:child relationship
            # parent2child lookup table
            if parent_node not in p2c_table:
                p2c_table[parent_node] = [child_node[0]]
            else:
                p2c_table[parent_node].append(child_node[0])
                
            #child2parent lookup table
            if child_node[0] not in c2p_table:
                c2p_table[child_node[0]] = [parent_node]
            else:
                c2p_table[child_node[0]].append(parent_node)
                
            # map parent/child node to a node<->id
            if parent_node not in node2id:
                p_id = i
                node2id[parent_node] = p_id
                id2node[p_id] = parent_node
                i+=1
            else:
                p_id = node2id[parent_node]
                
            if child_node[0] not in node2id:
                c_id = i
                node2id[child_node[0]] = c_id
                id2node[c_id] = child_node[0]      
                i+=1
            else:
                c_id = node2id[child_node[0]]

    pi_parents = set(p2c_table.keys())        
    T_leaves = (c2p_table.keys() - p2c_table.keys()) 
    N_all_nodes = pi_parents.union(T_leaves)
    
    return p2c_table, c2p_table, node2id, id2node, list(pi_parents), list(T_leaves), list(N_all_nodes)

In [39]:
def hierarchy2graph(p2c_table, node2id):

    edges = []
    for parent, children in p2c_table.items():
        p_id = node2id[parent]
        for child in children:
            c_id = node2id[child]
            edges.append((p_id, c_id))
    vertices = [k for k, v in node2id.items()]
    g = ig.Graph(n=len(node2id), edges=edges, directed=True, vertex_attrs={"name": vertices})
    return g

In [40]:
p2c, c2p, n2i, i2n, pi, T, N = lookup_table("swiki/data/cat_hier.txt", subset = False)
p2c_s, c2p_s, n2i_s, i2n_s, pi_s, T_s, N_s = lookup_table("swiki/data/cat_hier.txt", subset = 50)

65333it [00:00, 123442.02it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 99911.96it/s]


In [41]:
len(pi), len(T), len(N)

(13808, 36504, 50312)

In [42]:
g = hierarchy2graph(p2c_s, n2i_s)

In [43]:
print(g)

IGRAPH DN-- 51 50 --
+ attr: name (v)
+ edges (vertex names):
2143406->2156813, 2143406->2322682, 2143406->143406, 2143406->2255744,
2143406->2235965, 2156813->2440809, 2156813->2159645, 2156813->2267844,
2156813->2271677, 2156813->2152343, 2156813->1008038, 2156813->2310019,
2156813->2266243, 2156813->2426918, 2156813->2334805, 2156813->2132486,
2156813->2078122, 2156813->2057861, 2156813->2271281, 2156813->1012550,
2156813->2042688, 2156813->2373246, 2156813->2244017, 2156813->156813,
2322682->2013402, 2322682->2037186, 2322682->2341442, 2322682->2031546,
2322682->2305909, 2322682->2205397, 2322682->1112766, 2322682->2230320,
2322682->2402750, 2322682->2326915, 2322682->2242317, 2322682->2418165,
2322682->2196567, 2322682->2256232, 2322682->2127748, 2322682->2305967,
2322682->2436982, 2322682->2207289, 2322682->2386980, 2322682->2151145,
2322682->2080615, 2322682->322682, 2255744->2312526, 2255744->2324773,
2255744->2127372, 2255744->2091567


In [44]:
layout = g.layout("kk")
g.vs["label"] = g.vs["name"]
ig.plot(g, layout = layout)

In [47]:
X_train, x_test = train_test_split(train_data, test_size=0.3, random_state=42)

In [51]:
X_train.shape

(319820, 2085164)